In [ ]:
# from utils import process_ns3_csv
# input_file = 'D:/zhuomian/file/弱网中鲁棒性的恶意节点检测/exp/dataset/20241106_testTX_constantUniform_simtime-100_num_nodes-50_BHradio-0_SFradio-0_fieldLength-20_uniform-1/MonitorSnifferRx.csv'
# output_file = 'D:/zhuomian/file/弱网中鲁棒性的恶意节点检测/exp/dataset/20241106_testTX_constantUniform_simtime-100_num_nodes-50_BHradio-0_SFradio-0_fieldLength-20_uniform-1/ProcessedMonitorSnifferRx.csv'
# log_file = 'D:/zhuomian/file/弱网中鲁棒性的恶意节点检测/exp/dataset/20241106_testTX_constantUniform_simtime-100_num_nodes-50_BHradio-0_SFradio-0_fieldLength-20_uniform-1/log1106_uniform_20#%20_seed_12345.txt'
# process_ns3_csv(input_file, output_file, log_file)

In [ ]:
from utils import load_data
import numpy as np
import models 

In [ ]:
data_file = 'D:/zhuomian/file/弱网中鲁棒性的恶意节点检测/exp/dataset/20241106_testTX_constantUniform_simtime-100_num_nodes-50_BHradio-0_SFradio-0_fieldLength-20_uniform-1/ProcessedMonitorSnifferRx.csv'

In [ ]:
# np.set_printoptions(threshold=np.inf)  # 这将打印所有内容，去掉折叠
# data_type==1: ns3数据集；data_type==2：wwsn数据集
processed_data, adj, T, group_num = load_data(1,data_file)

In [ ]:
# 设置numpy的打印选项，显示更多行列


# 打印节点特征（processed_data.x）
# print(processed_data)
# print(adj[12])
# print(T[12])

In [ ]:
# 打印边连接关系（processed_data.edge_index）
# print(processed_data[-1].edge_index.numpy().size)

In [ ]:
# 打印节点标签（processed_data.y）
# print(processed_data[0].y.numpy())
# print(processed_data[-1].x[:,0:8].numpy().size)
# print(processed_data[0].y.numpy())
# print(processed_data[12].edge_index)
# print(processed_data[12].edge_weights)
# print(processed_data[12].x)

In [ ]:
train_model = models.HypergraphModel(8, 3)
print(train_model)

In [ ]:
import time
import torch.optim as optim
from utils import accuracy, auc, precision, recall, f1_score
import torch.nn.functional as F
import torch
criteria = torch.nn.CrossEntropyLoss()
learning_rate = 0.001
weight_decay = 5e-4
optimizer = optim.Adam(train_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
acc_measure = accuracy
def train(epoch):
    t = time.time()
    train_model.train()
    
    
    optimizer.zero_grad()
    # print(processed_data[0].x[0:8].shape)
    # print(processed_data[0].edge_weights.shape)
    # print(processed_data[0].edge_index.shape)
    for i in range(group_num):
        print(i)
        output = train_model(processed_data[i].x[:, 0:8], processed_data[i].edge_index, 
                                 processed_data[i].edge_weights, processed_data[i].x[:, 8:11], adj[i], T[i])
        # print(output)
        labels = processed_data[i].y
        loss_train = criteria(output, labels)
        # print(output[:, 1])
        acc_train = acc_measure(output, labels)
        
        loss_train.backward(retain_graph=True)
        optimizer.step()
        
        train_model.eval()
        output = train_model(processed_data[i].x[:, 0:8], processed_data[i].edge_index, 
                                 processed_data[i].edge_weights, processed_data[i].x[:, 8:11], adj[i], T[i])
        # print(output)
        loss_val = criteria(output, labels)
        acc_val = acc_measure(output, labels)
        auc_val = auc(output, labels)
        per_val = precision(output, labels)
        recall_val = recall(output, labels)
        f1_val = f1_score(output, labels)
        print(
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'loss_val: {:.4f}'.format(loss_val.item()),
              'acc_val: {:.4f}'.format(acc_val.item()),
            'auc_val: {:.4f}'.format(auc_val),
            'per_val: {:.4f}'.format(per_val),
            'recall_val: {:.4f}'.format(recall_val),
            'f1_val: {:.4f}'.format(f1_val),
              'time: {:.4f}s'.format(time.time() - t))
    return loss_val.item()

In [ ]:
def test():
    train_model.eval()
    output = train_model(processed_data[group_num-1].x[:, 0:8], processed_data[group_num-1].edge_index, 
                             processed_data[group_num-1].edge_weights, processed_data[group_num-1].x[:, 8:11], adj[group_num-1], T[group_num-1])
    labels = processed_data[group_num-1].y
    
    loss_test = criteria(output, labels)
    acc_test = acc_measure(output, labels)
    auc_test = auc(output, labels)
    per_test = precision(output, labels)
    recall_test = recall(output, labels)
    f1_test = f1_score(output, labels)
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()),
         'auc_test: {:.4f}'.format(auc_test),
            'per_test: {:.4f}'.format(per_test),
            'recall_test: {:.4f}'.format(recall_test),
            'f1_test: {:.4f}'.format(f1_test))
    return acc_test.item()

In [ ]:
# torch.autograd.set_detect_anomaly(True)
num_epochs = 1000
early_stopping = 30
val_watch = []
t_total = time.time()
for epoch in range(num_epochs):
    val = train(epoch)  # 训练并获得结果
    if val is None:  # 检查是否返回了None值
        print(f"Warning: train(epoch) returned None for epoch {epoch}. Skipping this iteration.")
        continue
    
    val_watch.append(val)  # 将结果添加到val_watch
    test()
    if epoch > early_stopping and val_watch[-1] > np.mean(val_watch[-(early_stopping + 1):-1]):
        print("Early stopping...")
        break
    
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))